In [1]:
import pickle
import numpy as np
import os
import json
from sklearn.metrics import f1_score, accuracy_score

# MAML

Note that the code below implements loading of data from new experiments. We also provide pickled data from running our experiments -- loading of this data is at the end of this notebook.

In [2]:
model_name = 'MAML'

In [3]:
DATA_PATH = os.path.join('..', '..', 'results', 'stability')
FACTORS = os.path.join(DATA_PATH, 'factors', 'predictions')
GOLDEN = os.path.join(DATA_PATH, 'golden', 'predictions', 'golden_model')

In [4]:
results = []
failed = 0
all = 0 

for split in os.listdir(GOLDEN):
    split_path = os.path.join(GOLDEN, split)
    for label in os.listdir(split_path):
        label_path = os.path.join(split_path, label)
        if os.path.isdir(label_path):
            for run in os.listdir(label_path):
                run_path = os.path.join(label_path, run)
                if os.path.isdir(run_path):
                    for evaluation in os.listdir(os.path.join(run_path, model_name)):
                        evaluation_path = os.path.join(run_path, model_name, evaluation)
                        if os.path.isdir(evaluation_path):
                            with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                data = json.load(file)
                            score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                            results.append(score)
                            if score < 0.5:
                                failed += 1
                            all += 1

In [5]:
np.mean(results) * 100

79.93851098599407

In [6]:
np.std(results) * 100

2.342405353448409

In [7]:
print(f"Failed percentage of runs: {failed / all * 100}%")

Failed percentage of runs: 0.18%


## Load Data

#### Data Split

In [8]:
data_split_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(FACTORS, 'data_split')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for run in os.listdir(label_path):
                    run_path = os.path.join(label_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                data_split_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    data_split_results['failed'] += 1
                                data_split_results['all'] += 1

In [9]:
len(data_split_results['results']), data_split_results['failed']

(10000, 0)

#### Label Selection

In [10]:
label_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(FACTORS, 'label_selection')
                                
for label in os.listdir(factor_path):
    label_path = os.path.join(factor_path, label)
    if label.startswith('label_') and os.path.isdir(label_path):
        for split in os.listdir(label_path):
            split_path = os.path.join(label_path, split)
            if split.startswith('split_') and os.path.isdir(split_path):
                for run in os.listdir(split_path):
                    run_path = os.path.join(split_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                label_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    label_results['failed'] += 1
                                label_results['all'] += 1

In [11]:
len(label_results['results']), label_results['failed']

(10000, 6)

#### Choice of Adaptation Data

In [12]:
adaptation_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(FACTORS, 'model_adaptation')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for run in os.listdir(label_path):
                    run_path = os.path.join(label_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                adaptation_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    adaptation_results['failed'] += 1
                                adaptation_results['all'] += 1

In [13]:
len(adaptation_results['results']), adaptation_results['failed']

(10000, 1)

##### Stable

In [14]:
stable_adaptation_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(DATA_PATH, 'adaptation_stable', 'predictions', 'model_adaptation')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for run in os.listdir(label_path):
                    run_path = os.path.join(label_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                stable_adaptation_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    stable_adaptation_results['failed'] += 1
                                stable_adaptation_results['all'] += 1

In [15]:
len(stable_adaptation_results['results']), stable_adaptation_results['failed']

(10000, 0)

In [16]:
stable_adaptation_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(DATA_PATH, 'adaptation_stable', 'predictions', 'model_adaptation')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for run in os.listdir(label_path):
                    run_path = os.path.join(label_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                adaptation_number = int(evaluation.split('_')[1])
                                if adaptation_number > 10:
                                    continue
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                stable_adaptation_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    stable_adaptation_results['failed'] += 1
                                stable_adaptation_results['all'] += 1

In [17]:
len(stable_adaptation_results['results']), stable_adaptation_results['failed']

(10000, 0)

##### Unstable

In [18]:
unstable_adaptation_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(DATA_PATH, 'adaptation_unstable', 'predictions', 'model_adaptation')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for run in os.listdir(label_path):
                    run_path = os.path.join(label_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                unstable_adaptation_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    unstable_adaptation_results['failed'] += 1
                                unstable_adaptation_results['all'] += 1

In [19]:
len(unstable_adaptation_results['results']), unstable_adaptation_results['failed']

(10000, 1)

In [20]:
unstable_adaptation_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(DATA_PATH, 'adaptation_unstable', 'predictions', 'model_adaptation')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for run in os.listdir(label_path):
                    run_path = os.path.join(label_path, run)
                    if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                        for evaluation in os.listdir(os.path.join(run_path, model_name)):
                            evaluation_path = os.path.join(run_path, model_name, evaluation)
                            if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                adaptation_number = int(evaluation.split('_')[1])
                                if adaptation_number > 10:
                                    continue
                                with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                    data = json.load(file)
                                score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                split_number = int(split.split('_')[1])
                                label_number = int(label.split('_')[1])
                                run_number = int(run.split('_')[1])
                                adaptation_number = int(evaluation.split('_')[1])
                                
                                unstable_adaptation_results['results'].append({
                                    'score': score,
                                    'split': split_number,
                                    'label': label_number,
                                    'run': run_number,
                                    'adaptation': adaptation_number
                                })
                                if score < 0.5:
                                    unstable_adaptation_results['failed'] += 1
                                unstable_adaptation_results['all'] += 1

In [21]:
len(unstable_adaptation_results['results']), unstable_adaptation_results['failed']

(10000, 1)

#### Initialisation of Model

In [22]:
initialisation_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(FACTORS, 'model_initialisation')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for initialisation in os.listdir(label_path):
                    initialisation_path = os.path.join(label_path, initialisation)
                    if initialisation.startswith('init_') and os.path.isdir(initialisation_path):
                        for run in os.listdir(initialisation_path):
                            run_path = os.path.join(initialisation_path, run)
                            if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                                for evaluation in os.listdir(os.path.join(run_path, model_name)):
                                    evaluation_path = os.path.join(run_path, model_name, evaluation)
                                    if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                        with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                            data = json.load(file)
                                        score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                        split_number = int(split.split('_')[1])
                                        label_number = int(label.split('_')[1])
                                        run_number = int(run.split('_')[1])
                                        adaptation_number = int(evaluation.split('_')[1])
                                        initialisation_number = int(initialisation.split('_')[1])
                                
                                        initialisation_results['results'].append({
                                            'score': score,
                                            'split': split_number,
                                            'label': label_number,
                                            'run': run_number,
                                            'adaptation': adaptation_number,
                                            'initialisation': initialisation_number
                                        })
                                        if score < 0.5:
                                            initialisation_results['failed'] += 1
                                        initialisation_results['all'] += 1

In [23]:
len(initialisation_results['results']), initialisation_results['failed']

(20000, 15)

#### Order of Train Data

In [24]:
order_results = {
    'results': [],
    'failed': 0,
    'all': 0
}

factor_path = os.path.join(FACTORS, 'data_order')

for split in os.listdir(factor_path):
    split_path = os.path.join(factor_path, split)
    if split.startswith('split_') and os.path.isdir(split_path):
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if label.startswith('label_') and os.path.isdir(label_path):
                for order in os.listdir(label_path):
                    order_path = os.path.join(label_path, order)
                    if order.startswith('data_order_') and os.path.isdir(order_path):
                        for run in os.listdir(order_path):
                            run_path = os.path.join(order_path, run)
                            if run.startswith('run_') and os.path.isdir(run_path) and os.path.exists(os.path.join(run_path, model_name)):  
                                for evaluation in os.listdir(os.path.join(run_path, model_name)):
                                    evaluation_path = os.path.join(run_path, model_name, evaluation)
                                    if evaluation.startswith('evaluation_') and os.path.isdir(evaluation_path):
                                        with open(os.path.join(evaluation_path, 'results.json'), 'r') as file:
                                            data = json.load(file)
                                        score = f1_score(np.array(data['predictions'][0]), np.array(data['predictions'][1]), average='macro')
                                
                                        split_number = int(split.split('_')[1])
                                        label_number = int(label.split('_')[1])
                                        run_number = int(run.split('_')[1])
                                        adaptation_number = int(evaluation.split('_')[1])
                                        order_number = int(order.split('_')[2])
                                
                                        order_results['results'].append({
                                            'score': score,
                                            'split': split_number,
                                            'label': label_number,
                                            'run': run_number,
                                            'adaptation': adaptation_number,
                                            'order': order_number
                                        })
                                        if score < 0.5:
                                            order_results['failed'] += 1
                                        order_results['all'] += 1

In [25]:
len(order_results['results']), order_results['failed']

(20000, 6)

## Compare Factors

### Aggregation by investigated factor

In this part we use the aggreagtion by the main investigated factor in following way:
- select runs where the value of factors only differ in the investigated factor (non-investigated factors have the same value; investigated has 10 values)
- calculate mean and standard deviation across the values of investigated factor
- results in ~10 000 values of mean and standard deviation
- calculate the final performance values as a mean of the pre-calculated many mean values
- calculate the instability of factor by calculating mean of the pre-calculated standard deviations

#### Data Split

In [52]:
data_split_by_other_factors = {}
overall_score = []

for result in data_split_results['results']:
    # key = f"split_{value['split']}-label_{value['label']}-run_{value['run']}-adaptation_{value['adaptation']}"
    key = f"label_{result['label']}-run_{result['run']}-adaptation_{result['adaptation']}"
    score = result['score'] * 100
    if data_split_by_other_factors.get(key, None) is None:
        data_split_by_other_factors[key] = [score]
    else:
        data_split_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(80.13179787985224, 0.960591117423806, 67.70128025238081, 82.80579181852798)

In [53]:
aggregated_data_split = {'mean': [], 'std': []}

for _, factor_value in data_split_by_other_factors.items():
    aggregated_data_split['mean'].append(np.mean(factor_value))
    aggregated_data_split['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_data_split['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_data_split['std'])}")
print(f"Other factors deviation: {np.std(aggregated_data_split['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_data_split['std'])}")

Investigated factor mean: 80.13179787985224
Investigated factor deviation: 0.846861446560856
Other factors deviation: 0.26271632755125723
Variability of factor deviation: 0.3695147039568942


In [54]:
print(f"Failed percentage of runs: {data_split_results['failed'] / data_split_results['all'] * 100}%")

Failed percentage of runs: 0.0%


#### Label Selection

In [55]:
label_by_other_factors = {}
overall_score = []

for result in label_results['results']:
    # key = f"split_{value['split']}-label_{value['label']}-run_{value['run']}-adaptation_{value['adaptation']}"
    key = f"split_{result['split']}-run_{result['run']}-adaptation_{result['adaptation']}"
    score = result['score'] * 100
    if label_by_other_factors.get(key, None) is None:
        label_by_other_factors[key] = [score]
    else:
        label_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(79.91335822496757, 1.7019957290123289, 30.69186223712283, 82.75265996281263)

In [56]:
aggregated_label = {'mean': [], 'std': []}

for _, factor_value in label_by_other_factors.items():
    aggregated_label['mean'].append(np.mean(factor_value))
    aggregated_label['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_label['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_label['std'])}")
print(f"Other factors deviation: {np.std(aggregated_label['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_label['std'])}")

Investigated factor mean: 79.91335822496755
Investigated factor deviation: 1.1791870196636256
Other factors deviation: 0.45101752903065
Variability of factor deviation: 1.1414423431519054


In [57]:
print(f"Failed percentage of runs: {label_results['failed'] / label_results['all'] * 100}%")

Failed percentage of runs: 0.06%


#### Choice of Adaptation Data

In [58]:
adaptation_by_other_factors = {}
overall_score = []

for result in adaptation_results['results']:
    # key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}-adaptation_{result['adaptation']}"
    key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}"
    score = result['score'] * 100
    if adaptation_by_other_factors.get(key, None) is None:
        adaptation_by_other_factors[key] = [score]
    else:
        adaptation_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(79.96216902291471, 1.245792306016174, 44.53243156871679, 82.68702082537425)

In [59]:
aggregated_adaptation = {'mean': [], 'std': []}

for _, factor_value in adaptation_by_other_factors.items():
    aggregated_adaptation['mean'].append(np.mean(factor_value))
    aggregated_adaptation['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_adaptation['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_adaptation['std'])}")
print(f"Other factors deviation: {np.std(aggregated_adaptation['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_adaptation['std'])}")

Investigated factor mean: 79.96216902291471
Investigated factor deviation: 0.4702532002912076
Other factors deviation: 1.0362507243324788
Variability of factor deviation: 0.5069958911720958


In [60]:
print(f"Failed percentage of runs: {adaptation_results['failed'] / adaptation_results['all'] * 100}%")

Failed percentage of runs: 0.01%


##### Stable

In [61]:
stable_adaptation_by_other_factors = {}
overall_score = []

for result in stable_adaptation_results['results']:
    # key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}-adaptation_{result['adaptation']}"
    key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}"
    score = result['score'] * 100
    if stable_adaptation_by_other_factors.get(key, None) is None:
        stable_adaptation_by_other_factors[key] = [score]
    else:
        stable_adaptation_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(80.18194322090837, 1.0018677723819405, 67.10120180844963, 82.79030958742395)

In [62]:
aggregated_stable_adaptation = {'mean': [], 'std': []}

for _, factor_value in stable_adaptation_by_other_factors.items():
    aggregated_stable_adaptation['mean'].append(np.mean(factor_value))
    aggregated_stable_adaptation['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_stable_adaptation['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_stable_adaptation['std'])}")
print(f"Other factors deviation: {np.std(aggregated_stable_adaptation['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_stable_adaptation['std'])}")

Investigated factor mean: 80.18194322090838
Investigated factor deviation: 0.16727928195009548
Other factors deviation: 0.9769734381367209
Variability of factor deviation: 0.14587520811682386


In [63]:
print(f"Failed percentage of runs: {stable_adaptation_results['failed'] / stable_adaptation_results['all'] * 100}%")

Failed percentage of runs: 0.0%


##### Unstable

In [64]:
unstable_adaptation_by_other_factors = {}
overall_score = []

for result in unstable_adaptation_results['results']:
    # key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}-adaptation_{result['adaptation']}"
    key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}"
    score = result['score'] * 100
    if unstable_adaptation_by_other_factors.get(key, None) is None:
        unstable_adaptation_by_other_factors[key] = [score]
    else:
        unstable_adaptation_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(79.96216902291471, 1.245792306016174, 44.53243156871679, 82.68702082537425)

In [65]:
aggregated_unstable_adaptation = {'mean': [], 'std': []}

for _, factor_value in unstable_adaptation_by_other_factors.items():
    aggregated_unstable_adaptation['mean'].append(np.mean(factor_value))
    aggregated_unstable_adaptation['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_unstable_adaptation['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_unstable_adaptation['std'])}")
print(f"Other factors deviation: {np.std(aggregated_unstable_adaptation['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_unstable_adaptation['std'])}")

Investigated factor mean: 79.96216902291471
Investigated factor deviation: 0.4702532002912076
Other factors deviation: 1.0362507243324788
Variability of factor deviation: 0.5069958911720958


In [66]:
print(f"Failed percentage of runs: {unstable_adaptation_results['failed'] / unstable_adaptation_results['all'] * 100}%")

Failed percentage of runs: 0.01%


#### Initialisation of Model

In [67]:
initialisation_by_other_factors = {}
overall_score = []

for result in initialisation_results['results']:
    key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}-adaptation_{result['adaptation']}"
    score = result['score'] * 100
    if initialisation_by_other_factors.get(key, None) is None:
        initialisation_by_other_factors[key] = [score]
    else:
        initialisation_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(79.98333781937309, 1.6675303758156068, 35.78085286642192, 82.85333506324773)

In [68]:
aggregated_initialisation = {'mean': [], 'std': []}

for _, factor_value in initialisation_by_other_factors.items():
    aggregated_initialisation['mean'].append(np.mean(factor_value))
    aggregated_initialisation['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_initialisation['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_initialisation['std'])}")
print(f"Other factors deviation: {np.std(aggregated_initialisation['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_initialisation['std'])}")

Investigated factor mean: 79.98333781937309
Investigated factor deviation: 0.6778194226254843
Other factors deviation: 0.8971914391031136
Variability of factor deviation: 1.2313674943653004


In [69]:
print(f"Failed percentage of runs: {initialisation_results['failed'] / initialisation_results['all'] * 100}%")

Failed percentage of runs: 0.075%


#### Order of Train Data

In [70]:
order_by_other_factors = {}
overall_score = []

for result in order_results['results']:
    key = f"split_{result['split']}-label_{result['label']}-run_{result['run']}-adaptation_{result['adaptation']}"
    score = result['score'] * 100
    if order_by_other_factors.get(key, None) is None:
        order_by_other_factors[key] = [score]
    else:
        order_by_other_factors[key].append(score)
    overall_score.append(score)
np.mean(overall_score), np.std(overall_score), np.min(overall_score), np.max(overall_score)

(79.57718227373387, 1.5399444414812988, 35.7990851396949, 82.74664354327439)

In [71]:
aggregated_order = {'mean': [], 'std': []}

for _, factor_value in order_by_other_factors.items():
    aggregated_order['mean'].append(np.mean(factor_value))
    aggregated_order['std'].append(np.std(factor_value))

print(f"Investigated factor mean: {np.mean(aggregated_order['mean'])}")
print(f"Investigated factor deviation: {np.mean(aggregated_order['std'])}")
print(f"Other factors deviation: {np.std(aggregated_order['mean'])}")
print(f"Variability of factor deviation: {np.std(aggregated_order['std'])}")

Investigated factor mean: 79.57718227373387
Investigated factor deviation: 1.0078957943548463
Other factors deviation: 0.8268150194809051
Variability of factor deviation: 0.8197267069772418


In [72]:
print(f"Failed percentage of runs: {order_results['failed'] / order_results['all'] * 100}%")

Failed percentage of runs: 0.03%


# Save Data

In [76]:
PICKLE_PATH = os.path.join('..', '..', 'pickled', 'sst2')

In [77]:
with open(os.path.join(PICKLE_PATH, 'MAML-data'), 'wb') as file:
    pickle.dump({
        'golden': results,
        'split': data_split_results,
        'label': label_results,
        'initialisation': initialisation_results,
        'order': order_results,
        'adaptation': adaptation_results,
        's_adaptation': stable_adaptation_results,
        'u_adaptation': unstable_adaptation_results,
    }, file)

In [ ]:
with open(os.path.join(PICKLE_PATH, 'MAML-data'), 'rb') as file:
    pickled = pickle.load(file)

results = pickled['golden']
data_split_results = pickled['split']
label_results = pickled['label']
initialisation_results = pickled['initialisation']
order_results = pickled['order']
adaptation_results = pickled['adaptation_results']
stable_adaptation_results = pickled['s_adaptation']
unstable_adaptation_results = pickled['u_adaptation']